<div style="text-align: center;">
<span style="font-size:45px;">Exploratory Data Analysis</span>
</div>

___

<div style="text-align: center;">
<span style="font-size:37px;">Ingest Datasets and First Analysis</span>
</div>

<div style="text-align: center;">At this initial stage, we will perform a validation of our datasets individually. The objective is to identify data inconsistencies, duplicate values, unexpected entries in specific columns, among other issues.</div>

### Allowance Events 

Since this is our source of truth, we will perform only minor consistency checks on the **allowance_events** dataset.

In [1]:
import pandas as pd
import json
from loguru import logger
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.max_colwidth', None)

In [2]:
with open('./datasets/allowance_events') as f:
    data = json.load(f)

allowance_events_df = pd.json_normalize(
    data,
    sep='_',  
    max_level=None
)

logger.info(f'Loaded allowance events data')

allowance_events_df.head()

2025-01-15 03:48:33.114 | INFO     | __main__:<module>:10 - Loaded allowance events data


,user_id,event_timestamp,event_name,allowance_scheduled_frequency,allowance_scheduled_day,allowance_amount
0,03c5ffe8-6cd2-434b-a1b2-c4006860d94c,2024-09-21 1:39:34,allowance.created,weekly,sunday,10
1,03c5ffe8-6cd2-434b-a1b2-c4006860d94c,2024-09-22 19:38:22,allowance.edited,weekly,sunday,10
2,03c5ffe8-6cd2-434b-a1b2-c4006860d94c,2024-09-22 19:38:22,allowance.edited,weekly,sunday,10
3,05384674-4af8-4f70-8d8f-90e19b0db971,2024-10-06 6:32:28,allowance.created,biweekly,friday,10
4,05384674-4af8-4f70-8d8f-90e19b0db971,2024-10-06 6:32:37,allowance.edited,weekly,friday,10


In [3]:
allowance_events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6208 entries, 0 to 6207
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   user_id                        6208 non-null   object
 1   event_timestamp                6208 non-null   object
 2   event_name                     6208 non-null   object
 3   allowance_scheduled_frequency  6208 non-null   object
 4   allowance_scheduled_day        6208 non-null   object
 5   allowance_amount               6208 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 291.1+ KB


We can observe that the dataset above does not contain any null values

In [4]:
allowance_events_df.nunique()

user_id                          2879
event_timestamp                  5191
event_name                          2
allowance_scheduled_frequency       4
allowance_scheduled_day            10
allowance_amount                   61
dtype: int64

We observed the presence of duplicate values in the dataset; however, this was expected since any updates are reflected in the data, providing us with a history of changes.

In [5]:
columns_to_check = ['event_name', 'allowance_scheduled_frequency', 'allowance_scheduled_day']

for column in columns_to_check:
    print(f'The types present in {column} are: {allowance_events_df[column].unique()}')

The types present in event_name are: ['allowance.created' 'allowance.edited']
The types present in allowance_scheduled_frequency are: ['weekly' 'biweekly' 'monthly' 'daily']
The types present in allowance_scheduled_day are: ['sunday' 'friday' 'thursday' 'saturday' 'first_day' 'fifteenth_day'
 'monday' 'daily' 'wednesday' 'tuesday']


We verified that there are no unexpected values in the fields analyzed.

In [6]:
last_updated_check = (allowance_events_df['event_timestamp'].unique()).max()
print(f'The last updated date is: {last_updated_check}')

The last updated date is: 2024-12-03 5:07:41


The latest update date aligns with the current date (2024-12-03).

In [7]:
current_date = datetime(2024, 12, 3)

def calculate_next_payment_event(day, frequency):
    if day == 'fifteenth_day':
        return 15
    elif day == 'first_day':
        return 1
    elif day == 'daily':
        return (current_date + timedelta(days=1)).day
    else:
        days_of_week = {
            'monday': 0,
            'tuesday': 1,
            'wednesday': 2,
            'thursday': 3,
            'friday': 4,
            'saturday': 5,
            'sunday': 6
        }
        target_day = days_of_week[day]
        current_weekday = current_date.weekday()
        days_until_next = (target_day - current_weekday + 7) % 7
        if days_until_next == 0:
            days_until_next = 7
        next_payment_date = current_date + timedelta(days=days_until_next)

        if frequency == 'biweekly':
            next_payment_date += timedelta(weeks=1)
        
        return next_payment_date.day
        

allowance_events_df['next_payment_event'] = allowance_events_df.apply(
    lambda row: calculate_next_payment_event(row['allowance_scheduled_day'], row['allowance_scheduled_frequency']), axis=1
)

In [8]:
allowance_events_df['event_timestamp'] = pd.to_datetime(allowance_events_df['event_timestamp'])
allowance_events_df['event_timestamp'] = allowance_events_df['event_timestamp'].astype(int) // 10**9
allowance_events_df.head()

,user_id,event_timestamp,event_name,allowance_scheduled_frequency,allowance_scheduled_day,allowance_amount,next_payment_event
0,03c5ffe8-6cd2-434b-a1b2-c4006860d94c,1726882774,allowance.created,weekly,sunday,10,8
1,03c5ffe8-6cd2-434b-a1b2-c4006860d94c,1727033902,allowance.edited,weekly,sunday,10,8
2,03c5ffe8-6cd2-434b-a1b2-c4006860d94c,1727033902,allowance.edited,weekly,sunday,10,8
3,05384674-4af8-4f70-8d8f-90e19b0db971,1728196348,allowance.created,biweekly,friday,10,13
4,05384674-4af8-4f70-8d8f-90e19b0db971,1728196357,allowance.edited,weekly,friday,10,6


Since the dataset is considered the source of truth and the current date is 2024-12-03, we can calculate the next_payment_event field. This will assist in further investigations and provide better insight into potential inconsistencies in the backend datasets.

In [9]:
allowance_events_df_last_updated = allowance_events_df.sort_values(by='event_timestamp') \
                                        .drop_duplicates(subset=['user_id'], keep='last')

allowance_events_df_last_updated.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2879 entries, 1701 to 1233
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   user_id                        2879 non-null   object
 1   event_timestamp                2879 non-null   int64 
 2   event_name                     2879 non-null   object
 3   allowance_scheduled_frequency  2879 non-null   object
 4   allowance_scheduled_day        2879 non-null   object
 5   allowance_amount               2879 non-null   int64 
 6   next_payment_event             2879 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 179.9+ KB


Above, we performed a data cleaning step to retain only the latest updates for each 'user_id' present.

In [10]:
allowance_events_df[['allowance_scheduled_frequency', 'allowance_scheduled_day']].drop_duplicates().sort_values(by='allowance_scheduled_frequency')

,allowance_scheduled_frequency,allowance_scheduled_day
384,biweekly,tuesday
3,biweekly,friday
8,biweekly,thursday
9,biweekly,saturday
37,biweekly,monday
279,biweekly,sunday
87,biweekly,wednesday
75,daily,daily
19,monthly,first_day
30,monthly,fifteenth_day


Above, we can confirm the absence of inconsistencies in the expected data regarding frequency and date.

### Allowance Backend 

In [11]:
allowance_backend_df = pd.read_csv('./datasets/allowance_backend_table')

logger.info('Loaded Allowance Backend data')
allowance_backend_df.head()

2025-01-15 03:48:48.723 | INFO     | __main__:<module>:3 - Loaded Allowance Backend data


,uuid,creation_date,frequency,day,updated_at,next_payment_day,status
0,30f4e25e-3e37-462e-8c3c-42f24f54350f,1724838709,monthly,fifteenth_day,2024-10-15T08:00:41.445627563Z,15,enabled
1,6da398ad-079d-49b9-8668-6d7ce4d22683,1724659829,monthly,fifteenth_day,1724659829,15,enabled
2,2d30fe2d-6c32-4b8a-a19b-906184f64f62,1731309159,monthly,fifteenth_day,1731309159,15,enabled
3,c30180f3-b35c-470c-b25a-f12ec4ce3340,1725414982,monthly,first_day,2024-10-01T08:00:37.516552581Z,1,enabled
4,9f5768db-e953-4057-9fa5-264c46293fbd,1726267511,biweekly,friday,2024-10-02T02:33:10.97923183Z,11,disabled


<span style="font-family:Arial; font-size:16px;">Above, we can observe the first inconsistency in the data. In the **updated_at** field, we can see that the data has different types when updated in the database, indicating a potential flaw in the process design. Upon closer inspection, we notice that when the **creation_date** field is equal to **updated_at**, the **updated_at** field takes on the timestamp value. However, when they differ, the **updated_at** field assumes a datetime value.

<span style="font-family:Arial; font-size:16px;">Note: When we mention that the field values diverge, we mean that when converting the creation_date field from timestamp to datetime, the output does not match the **updated_at** field.

</span>

In [12]:
allowance_backend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2882 entries, 0 to 2881
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   uuid              2882 non-null   object
 1   creation_date     2882 non-null   int64 
 2   frequency         2882 non-null   object
 3   day               2882 non-null   object
 4   updated_at        2882 non-null   object
 5   next_payment_day  2882 non-null   int64 
 6   status            2882 non-null   object
dtypes: int64(2), object(5)
memory usage: 157.7+ KB


In [13]:
allowance_backend_df.nunique()

uuid                2882
creation_date       2880
frequency              4
day                   10
updated_at          2882
next_payment_day      31
status                 2
dtype: int64

In [14]:
columns_to_check = ['frequency', 'day', 'next_payment_day', 'status']

for column in columns_to_check:
    print(f'The types present in {column} are: {(allowance_backend_df[column].unique())}')

The types present in frequency are: ['monthly' 'biweekly' 'weekly' 'daily']
The types present in day are: ['fifteenth_day' 'first_day' 'friday' 'monday' 'tuesday' 'thursday'
 'saturday' 'sunday' 'daily' 'wednesday']
The types present in next_payment_day are: [15  1 11 16 10  2  5  6  7 28 29  3  8 25  4 18 13 12 20 22 30  9 27 21
 26 19 23 24 31 14 17]
The types present in status are: ['enabled' 'disabled']


In [15]:
allowance_backend_df[['frequency', 'day']].drop_duplicates().sort_values(by='frequency')

,frequency,day
156,biweekly,saturday
104,biweekly,wednesday
4,biweekly,friday
5,biweekly,monday
8,biweekly,tuesday
37,biweekly,sunday
15,biweekly,thursday
101,daily,daily
0,monthly,fifteenth_day
3,monthly,first_day


As previously checked in the *allowance_events* dataset, we can also observe that the *allowance_backend* dataset does not contain any null or duplicate values, and it has the expected values in the investigated columns.

In [16]:
df_timestamp = allowance_backend_df[~(allowance_backend_df['updated_at'].str.contains('T'))]
df_timestamp['updated_at'] = df_timestamp['updated_at'].astype(int)
df_datetime = allowance_backend_df[(allowance_backend_df['updated_at'].str.contains('T'))]
df_datetime['updated_at'] = pd.to_datetime(df_datetime['updated_at'])
df_datetime['updated_at'] = df_datetime['updated_at'].astype(int) // 10**9

allowance_backend_df = pd.concat([df_timestamp, df_datetime])

allowance_backend_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2882 entries, 1 to 2881
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   uuid              2882 non-null   object
 1   creation_date     2882 non-null   int64 
 2   frequency         2882 non-null   object
 3   day               2882 non-null   object
 4   updated_at        2882 non-null   int64 
 5   next_payment_day  2882 non-null   int64 
 6   status            2882 non-null   object
dtypes: int64(3), object(4)
memory usage: 180.1+ KB


/tmp/ipykernel_302435/3564889471.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_timestamp['updated_at'] = df_timestamp['updated_at'].astype(int)
/tmp/ipykernel_302435/3564889471.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_datetime['updated_at'] = pd.to_datetime(df_datetime['updated_at'])
/tmp/ipykernel_302435/3564889471.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [17]:
allowance_backend_df.head()

,uuid,creation_date,frequency,day,updated_at,next_payment_day,status
1,6da398ad-079d-49b9-8668-6d7ce4d22683,1724659829,monthly,fifteenth_day,1724659829,15,enabled
2,2d30fe2d-6c32-4b8a-a19b-906184f64f62,1731309159,monthly,fifteenth_day,1731309159,15,enabled
5,35d3d7c2-2a05-4eae-bdf2-7896e611bbc6,1725453772,biweekly,monday,1725453772,16,enabled
6,9295f1b0-c1cf-4dca-b720-84146395e058,1725501541,biweekly,monday,1725501541,16,enabled
7,c398bb9d-8ba5-4f31-9dd2-97d09a289360,1725453807,biweekly,monday,1725453807,16,enabled


Cleaning step on the **updated_at** field, converting all values to the timestamp type, which will enable future analyses of the data.

### Payment Scheduled Backend

In [18]:
payment_schedule_df = pd.read_csv('./datasets/payment_schedule_backend_table')

logger.info('Loaded Payment Schedule Backend data')

payment_schedule_df.head()

2025-01-15 03:49:05.153 | INFO     | __main__:<module>:3 - Loaded Payment Schedule Backend data


,user_id,payment_date
0,061d8039-b332-40ca-b906-3514b06411c2,22
1,0ca92d01-feed-42e5-b90c-299b615f9d3f,22
2,160bcd2d-3ab0-4a60-9bfa-ccfcc1ee17e4,22
3,1649fee7-c424-4fb7-ac84-db36e15e042a,22
4,1d647703-874d-4744-b4de-6a5212518aa8,22


In [19]:
payment_schedule_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2139 entries, 0 to 2138
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       2139 non-null   object
 1   payment_date  2139 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 33.5+ KB


In [20]:
payment_schedule_df.nunique()

user_id         2126
payment_date      31
dtype: int64

In [21]:
grouped  = payment_schedule_df.groupby('user_id').size()
print(f"The number of users without a unique registration in the 'payment_scheduled_backend' table {grouped[grouped > 1].shape[0]}")

The number of users without a unique registration in the 'payment_scheduled_backend' table 12


In [22]:
print(f"Users with multiple registrations in the 'payment_scheduled_backend' table")
grouped[grouped > 1].sort_values(ascending=False)

Users with multiple registrations in the 'payment_scheduled_backend' table


user_id
72e8fe3f-0dd9-44e3-8775-660bacb17310    3
172fe923-347e-4d0e-ba82-1e9969aaadde    2
42f61042-3d31-42ff-9f1a-508716069464    2
19d933a6-2810-4c08-b3d6-d68ef3c6eac3    2
72190913-1208-4333-aaf4-ea7d04dccacd    2
77cdda96-6e5b-47c1-b486-9f06da2c7372    2
9221dccf-44f3-446d-b3ce-19f9123ea627    2
abf5cf1e-e81a-4b5f-9d5b-649fa10d8f81    2
c091a91b-c6fd-44cb-8ac7-fe3fdbafccea    2
d299d804-0d05-48c8-ad62-601d54438e34    2
d769685e-9046-4ffe-a51d-829ec7f1d877    2
eab4d50f-2a17-4f07-b462-53be0e54f6c6    2
dtype: int64

Upon reviewing previous executions, we can observe that there are duplicate records in the investigated database, one of which is triplicated. This indicates a data inconsistency, as the database should only contain one record per user.

In [23]:
filter_grouped_payments = grouped[grouped > 1].index
payment_schedule_df[payment_schedule_df['user_id'].isin(filter_grouped_payments)].sort_values(by='user_id')

,user_id,payment_date
1575,172fe923-347e-4d0e-ba82-1e9969aaadde,11
1629,172fe923-347e-4d0e-ba82-1e9969aaadde,3
1498,19d933a6-2810-4c08-b3d6-d68ef3c6eac3,7
1178,19d933a6-2810-4c08-b3d6-d68ef3c6eac3,28
972,42f61042-3d31-42ff-9f1a-508716069464,6
1652,42f61042-3d31-42ff-9f1a-508716069464,20
1384,72190913-1208-4333-aaf4-ea7d04dccacd,4
1592,72190913-1208-4333-aaf4-ea7d04dccacd,11
2119,72e8fe3f-0dd9-44e3-8775-660bacb17310,14
239,72e8fe3f-0dd9-44e3-8775-660bacb17310,13


In [24]:
allowance_backend_df[allowance_backend_df['uuid'].isin(filter_grouped_payments)].sort_values(by='uuid')

,uuid,creation_date,frequency,day,updated_at,next_payment_day,status
2780,172fe923-347e-4d0e-ba82-1e9969aaadde,1726790868,biweekly,friday,1727401816,11,enabled
1130,19d933a6-2810-4c08-b3d6-d68ef3c6eac3,1732603216,weekly,saturday,1732987949,7,enabled
1119,42f61042-3d31-42ff-9f1a-508716069464,1730512210,weekly,wednesday,1731871730,20,enabled
1291,72190913-1208-4333-aaf4-ea7d04dccacd,1727446537,weekly,wednesday,1732694428,4,enabled
408,72e8fe3f-0dd9-44e3-8775-660bacb17310,1726083350,weekly,thursday,1732780824,5,enabled
193,77cdda96-6e5b-47c1-b486-9f06da2c7372,1729432012,monthly,first_day,1733040040,1,enabled
887,9221dccf-44f3-446d-b3ce-19f9123ea627,1727279570,weekly,friday,1730016036,1,enabled
2781,abf5cf1e-e81a-4b5f-9d5b-649fa10d8f81,1726790817,biweekly,friday,1727401809,11,enabled
2576,c091a91b-c6fd-44cb-8ac7-fe3fdbafccea,1728143296,weekly,friday,1732867236,6,enabled
2417,d299d804-0d05-48c8-ad62-601d54438e34,1722473464,weekly,sunday,1729616252,27,enabled


By comparing the duplicated data with the *allowance_backend* table, we can infer that the data inconsistency might be due to a possible update not being reflected in the *payment_scheduled* table, as both should have the *next_payment_day* and *payment_day* fields aligned. We can see that one of the duplicated values (or triplicated in one case) in the *payment_scheduled* table matches the value in the *allowance_backend* table, which supports the theory of a possible failure in the update process, leading to duplication.

In [25]:
print(f"The types present in 'payment_date' are: {(payment_schedule_df['payment_date'].unique())}")

The types present in 'payment_date' are: [22 18 16  2 13  8  9  1 31  6 28 30 24 27  5  4 23 19 26  7 11  3 20 29
 21 25 12 17 10 15 14]


Payment data is as expected, within the range of 1 to 31, representing the days of a month

---

<div style="text-align: center;">
<span style="font-size:37px;">Comparative analysis between datasets</span>
</div>

<div style="text-align: center;">We have started the exploratory analysis phase, where we can verify the data cohesion applied across the datasets, ensuring that the rules established for the process are being met.</div>

### Comparative Between Allowance Events & Allowance Backend

In [26]:
allowance_events_backend_df = pd.merge(allowance_events_df_last_updated, allowance_backend_df, left_on='user_id', right_on='uuid', how='inner')

logger.info('Merged Allowance Events and Allowance Backend data')

allowance_events_backend_df.head()

2025-01-15 03:49:42.007 | INFO     | __main__:<module>:3 - Merged Allowance Events and Allowance Backend data


,user_id,event_timestamp,event_name,allowance_scheduled_frequency,allowance_scheduled_day,allowance_amount,next_payment_event,uuid,creation_date,frequency,day,updated_at,next_payment_day,status
0,27602664-335c-4beb-9dd7-7c6942385b64,1721906789,allowance.created,weekly,friday,1,6,27602664-335c-4beb-9dd7-7c6942385b64,1721931989,weekly,friday,1722014536,26,disabled
1,78fea998-707b-4a93-b92c-d84362165097,1721907374,allowance.created,weekly,friday,1,6,78fea998-707b-4a93-b92c-d84362165097,1721932573,weekly,friday,1722790548,9,disabled
2,038c6ae6-ff6c-4b68-b33d-847a3ca50601,1721907403,allowance.created,biweekly,friday,2,13,038c6ae6-ff6c-4b68-b33d-847a3ca50601,1721932603,biweekly,friday,1722790563,16,disabled
3,febf7ed7-a2e0-40b8-899b-3fb511a91d32,1721907423,allowance.created,monthly,first_day,1,1,febf7ed7-a2e0-40b8-899b-3fb511a91d32,1721932623,monthly,first_day,1725179134,1,disabled
4,c5a7c5f7-6c1e-4ea0-919a-f7efdf4e7cc9,1721908557,allowance.edited,weekly,friday,1,6,c5a7c5f7-6c1e-4ea0-919a-f7efdf4e7cc9,1721931549,weekly,friday,1723390198,16,disabled


In [27]:
allowance_events_backend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2877 entries, 0 to 2876
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   user_id                        2877 non-null   object
 1   event_timestamp                2877 non-null   int64 
 2   event_name                     2877 non-null   object
 3   allowance_scheduled_frequency  2877 non-null   object
 4   allowance_scheduled_day        2877 non-null   object
 5   allowance_amount               2877 non-null   int64 
 6   next_payment_event             2877 non-null   int64 
 7   uuid                           2877 non-null   object
 8   creation_date                  2877 non-null   int64 
 9   frequency                      2877 non-null   object
 10  day                            2877 non-null   object
 11  updated_at                     2877 non-null   int64 
 12  next_payment_day               2877 non-null   int64 
 13  sta

In [28]:
allowance_events_backend_df.nunique()

user_id                          2877
event_timestamp                  2876
event_name                          2
allowance_scheduled_frequency       4
allowance_scheduled_day            10
allowance_amount                   45
next_payment_event                 15
uuid                             2877
creation_date                    2875
frequency                           4
day                                10
updated_at                       2194
next_payment_day                   31
status                              2
dtype: int64

Upon performing the join between the tables, we verified the absence of null values and duplicates, and confirmed that there are no users present in only one of the tables.

In [29]:
allowance_events_backend_enabled = allowance_events_backend_df[allowance_events_backend_df['status'] == 'enabled']
allowance_events_backend_enabled.head()

,user_id,event_timestamp,event_name,allowance_scheduled_frequency,allowance_scheduled_day,allowance_amount,next_payment_event,uuid,creation_date,frequency,day,updated_at,next_payment_day,status
6,17df1db7-0a6a-4661-90d6-949b601ab02b,1722000877,allowance.created,biweekly,monday,20,16,17df1db7-0a6a-4661-90d6-949b601ab02b,1722026076,biweekly,monday,1722026076,5,enabled
7,fb835932-ac1e-45a7-8706-545805841d0c,1722015013,allowance.created,biweekly,friday,10,13,fb835932-ac1e-45a7-8706-545805841d0c,1722040213,biweekly,friday,1722040213,9,enabled
10,64dd086c-5c33-4af1-a621-0936f01a59bc,1722075730,allowance.created,biweekly,friday,20,13,64dd086c-5c33-4af1-a621-0936f01a59bc,1722100930,biweekly,friday,1722100930,9,enabled
15,431a62c4-30ae-405d-95d2-fb7b8284ac4f,1722144030,allowance.created,biweekly,thursday,5,12,431a62c4-30ae-405d-95d2-fb7b8284ac4f,1722169230,biweekly,thursday,1723104031,22,enabled
16,d93f143d-830e-4364-88c1-04cadd36b354,1722153754,allowance.created,biweekly,friday,20,13,d93f143d-830e-4364-88c1-04cadd36b354,1722178953,biweekly,friday,1732867254,13,enabled


In [30]:
allowance_events_backend_enabled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2114 entries, 6 to 2876
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   user_id                        2114 non-null   object
 1   event_timestamp                2114 non-null   int64 
 2   event_name                     2114 non-null   object
 3   allowance_scheduled_frequency  2114 non-null   object
 4   allowance_scheduled_day        2114 non-null   object
 5   allowance_amount               2114 non-null   int64 
 6   next_payment_event             2114 non-null   int64 
 7   uuid                           2114 non-null   object
 8   creation_date                  2114 non-null   int64 
 9   frequency                      2114 non-null   object
 10  day                            2114 non-null   object
 11  updated_at                     2114 non-null   int64 
 12  next_payment_day               2114 non-null   int64 
 13  status  

In [31]:
allowance_events_backend_enabled_ics = allowance_events_backend_enabled[(allowance_events_backend_enabled['allowance_scheduled_frequency'] != allowance_events_backend_enabled['frequency']) 
                                                                        | (allowance_events_backend_enabled['allowance_scheduled_day'] != allowance_events_backend_enabled['day'])]
allowance_events_backend_enabled_ics

,user_id,event_timestamp,event_name,allowance_scheduled_frequency,allowance_scheduled_day,allowance_amount,next_payment_event,uuid,creation_date,frequency,day,updated_at,next_payment_day,status
518,308b2b9c-d49d-4b2d-947c-5b2370da090f,1725892304,allowance.created,weekly,sunday,2,8,308b2b9c-d49d-4b2d-947c-5b2370da090f,1725917503,monthly,first_day,1732242533,1,enabled


In [32]:
incosistency_values = allowance_events_backend_enabled_ics['user_id'].tolist()
allowance_events_df[allowance_events_df['user_id'].isin(incosistency_values)].sort_values(by=['user_id', 'event_timestamp'], ascending=[True, True])

,user_id,event_timestamp,event_name,allowance_scheduled_frequency,allowance_scheduled_day,allowance_amount,next_payment_event
1713,308b2b9c-d49d-4b2d-947c-5b2370da090f,1725892304,allowance.created,weekly,sunday,2,8


1. First, we applied a filter to include only enabled values, as outlined in the proposed challenge resolution.

2. Next, we performed a check to ensure that the frequency and day fields are consistent across the datasets. As observed in previous executions, for user **308b2b9c-d49d-4b2d-947c-5b2370da090f**, there is an inconsistency in these fields in the *allowance_backend* table.

3. Upon checking our source of truth, we confirmed that no updates have been made for this user since their creation, which rules out the possibility of an update not being reflected. Therefore, we can assume that an error occurred during the data insertion into the backend table at the time of creation.

In [33]:
incosistency_payment = allowance_events_backend_enabled[(allowance_events_backend_enabled['next_payment_event'] != allowance_events_backend_enabled['next_payment_day'])]
incosistency_payment

,user_id,event_timestamp,event_name,allowance_scheduled_frequency,allowance_scheduled_day,allowance_amount,next_payment_event,uuid,creation_date,frequency,day,updated_at,next_payment_day,status
6,17df1db7-0a6a-4661-90d6-949b601ab02b,1722000877,allowance.created,biweekly,monday,20,16,17df1db7-0a6a-4661-90d6-949b601ab02b,1722026076,biweekly,monday,1722026076,5,enabled
7,fb835932-ac1e-45a7-8706-545805841d0c,1722015013,allowance.created,biweekly,friday,10,13,fb835932-ac1e-45a7-8706-545805841d0c,1722040213,biweekly,friday,1722040213,9,enabled
10,64dd086c-5c33-4af1-a621-0936f01a59bc,1722075730,allowance.created,biweekly,friday,20,13,64dd086c-5c33-4af1-a621-0936f01a59bc,1722100930,biweekly,friday,1722100930,9,enabled
15,431a62c4-30ae-405d-95d2-fb7b8284ac4f,1722144030,allowance.created,biweekly,thursday,5,12,431a62c4-30ae-405d-95d2-fb7b8284ac4f,1722169230,biweekly,thursday,1723104031,22,enabled
21,d38a4008-15f8-45f4-abf8-d1b69c054baa,1722184730,allowance.created,weekly,friday,10,6,d38a4008-15f8-45f4-abf8-d1b69c054baa,1722209930,weekly,friday,1722209930,2,enabled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2801,ec801e13-6cb8-4079-b4d7-12186ca9b7f5,1732992387,allowance.created,weekly,monday,5,9,ec801e13-6cb8-4079-b4d7-12186ca9b7f5,1733021187,weekly,monday,1733021187,2,enabled
2806,159b3774-5819-480b-bb87-27096e8f2a37,1733000152,allowance.edited,biweekly,tuesday,25,17,159b3774-5819-480b-bb87-27096e8f2a37,1733028799,biweekly,tuesday,1733028951,10,enabled
2807,ad335f49-b596-4673-be2d-a65f4dec0173,1733000169,allowance.edited,biweekly,tuesday,25,17,ad335f49-b596-4673-be2d-a65f4dec0173,1733028827,biweekly,tuesday,1733028968,10,enabled
2821,6b588731-dd54-4246-8f3f-1c0ebe812482,1733061439,allowance.edited,weekly,monday,1,9,6b588731-dd54-4246-8f3f-1c0ebe812482,1727603157,weekly,monday,1733090234,2,enabled


In [34]:
incosistency_payment.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1066 entries, 6 to 2835
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   user_id                        1066 non-null   object
 1   event_timestamp                1066 non-null   int64 
 2   event_name                     1066 non-null   object
 3   allowance_scheduled_frequency  1066 non-null   object
 4   allowance_scheduled_day        1066 non-null   object
 5   allowance_amount               1066 non-null   int64 
 6   next_payment_event             1066 non-null   int64 
 7   uuid                           1066 non-null   object
 8   creation_date                  1066 non-null   int64 
 9   frequency                      1066 non-null   object
 10  day                            1066 non-null   object
 11  updated_at                     1066 non-null   int64 
 12  next_payment_day               1066 non-null   int64 
 13  status  

In [35]:
incosistency_payment.nunique()

user_id                          1066
event_timestamp                  1066
event_name                          2
allowance_scheduled_frequency       3
allowance_scheduled_day             8
allowance_amount                   32
next_payment_event                 14
uuid                             1066
creation_date                    1065
frequency                           4
day                                 9
updated_at                        930
next_payment_day                   31
status                              1
dtype: int64

In [36]:
incosistency_payment[incosistency_payment['frequency'] == 'daily'].sort_values('updated_at', ascending = False).head()

,user_id,event_timestamp,event_name,allowance_scheduled_frequency,allowance_scheduled_day,allowance_amount,next_payment_event,uuid,creation_date,frequency,day,updated_at,next_payment_day,status
2771,d2ad566f-c59f-4b85-8525-8f8dd051db3a,1732950586,allowance.created,daily,daily,20,4,d2ad566f-c59f-4b85-8525-8f8dd051db3a,1732979386,daily,daily,1732979386,1,enabled
2770,990d4a0c-cede-4938-bdb1-fdbab1544989,1732950419,allowance.edited,daily,daily,29,4,990d4a0c-cede-4938-bdb1-fdbab1544989,1732907281,daily,daily,1732979219,1,enabled
2703,bca5caad-03c6-413e-a375-6b1e1b95d28f,1732786300,allowance.edited,daily,daily,20,4,bca5caad-03c6-413e-a375-6b1e1b95d28f,1731427231,daily,daily,1732815099,29,enabled
2687,1f6e46f0-ea51-4cca-8c98-517292c8f301,1732767819,allowance.edited,daily,daily,5,4,1f6e46f0-ea51-4cca-8c98-517292c8f301,1732751023,daily,daily,1732796617,29,enabled
2683,d0734842-1640-4b43-96ac-794490884d30,1732741314,allowance.created,daily,daily,1,4,d0734842-1640-4b43-96ac-794490884d30,1732770114,daily,daily,1732770114,29,enabled


In [37]:
timestamp = 1732979386

dt = datetime.fromtimestamp(timestamp)

print(dt)

2024-11-30 12:09:46


In [38]:
timestamp = 1732815099

dt = datetime.fromtimestamp(timestamp)

print(dt)

2024-11-28 14:31:39


- Here we arrive at perhaps the most important observation in our exploratory analysis so far. By comparing the values of the *next_payment_day_event* and *next_payment_day fields*, where next_payment_day_event is considered our source of truth, we observe a significant number of incorrect values in the payment field of the backend table. Upon further investigation into the cause of these inconsistencies, we found that the *allowance_backend* table only holds the most recent update made by the user, and its *next_payment_day* field is not updated as payments are made.

To explain this in more detail, let's review the above executions:

- For user **d2ad566f-c59f-4b85-8525-8f8dd051db3a**, from the first line in the output of the executions above, the value of updated_at is 1732979386, which corresponds to 2024-11-30. With daily as the frequency, we can observe that the *next_payment_day* field assumes the value of the following day, 2024-12-01.

- The same observation applies to user **bca5caad-03c6-413e-a375-6b1e1b95d28f**, where updated_at represents 2024-11-28, and daily is the frequency. Checking the *next_payment_day* field, we can see it assumes the value 2024-11-29.

In [39]:
incosistency_payment[(incosistency_payment['frequency'] == 'biweekly')].sort_values('updated_at', ascending = True).head()

,user_id,event_timestamp,event_name,allowance_scheduled_frequency,allowance_scheduled_day,allowance_amount,next_payment_event,uuid,creation_date,frequency,day,updated_at,next_payment_day,status
6,17df1db7-0a6a-4661-90d6-949b601ab02b,1722000877,allowance.created,biweekly,monday,20,16,17df1db7-0a6a-4661-90d6-949b601ab02b,1722026076,biweekly,monday,1722026076,5,enabled
7,fb835932-ac1e-45a7-8706-545805841d0c,1722015013,allowance.created,biweekly,friday,10,13,fb835932-ac1e-45a7-8706-545805841d0c,1722040213,biweekly,friday,1722040213,9,enabled
10,64dd086c-5c33-4af1-a621-0936f01a59bc,1722075730,allowance.created,biweekly,friday,20,13,64dd086c-5c33-4af1-a621-0936f01a59bc,1722100930,biweekly,friday,1722100930,9,enabled
27,51c38ec8-8463-40de-9727-93035d0e77ff,1722265765,allowance.edited,biweekly,friday,10,13,51c38ec8-8463-40de-9727-93035d0e77ff,1722290701,biweekly,friday,1722290964,9,enabled
28,177f5aa7-aec7-4f16-9e2d-34a0a4eea959,1722265778,allowance.edited,biweekly,friday,10,13,177f5aa7-aec7-4f16-9e2d-34a0a4eea959,1722290793,biweekly,friday,1722290978,9,enabled


- Reinforcing the previous analysis, we have established that today’s date is 2024-12-03, which is a Tuesday.

- Taking user **17df1db7-0a6a-4661-90d6-949b601ab02b** as an example, we observe that their *updated_at* column represents 2024-07-27, which was a Saturday. The expected payment day, given a biweekly frequency, would be the following Monday. However, we can see that the *next_payment_day* field shows the value 5, which confirms that the data in the backend allowance table only contains the most recent update made by the user, and the *next_payment_day* field is not updated in line with the current day.

### Comparative Between Allowance Backend & Payment Scheduled Backend

In [40]:
allowance_payment_df = pd.merge(allowance_backend_df, payment_schedule_df, left_on='uuid', right_on='user_id', how='inner')

logger.info('Merged Allowance Backend and Payment Schedule Backend data')

allowance_payment_df.head()

2025-01-15 03:50:09.424 | INFO     | __main__:<module>:3 - Merged Allowance Backend and Payment Schedule Backend data


,uuid,creation_date,frequency,day,updated_at,next_payment_day,status,user_id,payment_date
0,6da398ad-079d-49b9-8668-6d7ce4d22683,1724659829,monthly,fifteenth_day,1724659829,15,enabled,6da398ad-079d-49b9-8668-6d7ce4d22683,15
1,2d30fe2d-6c32-4b8a-a19b-906184f64f62,1731309159,monthly,fifteenth_day,1731309159,15,enabled,2d30fe2d-6c32-4b8a-a19b-906184f64f62,15
2,35d3d7c2-2a05-4eae-bdf2-7896e611bbc6,1725453772,biweekly,monday,1725453772,16,enabled,35d3d7c2-2a05-4eae-bdf2-7896e611bbc6,16
3,9295f1b0-c1cf-4dca-b720-84146395e058,1725501541,biweekly,monday,1725501541,16,enabled,9295f1b0-c1cf-4dca-b720-84146395e058,16
4,c398bb9d-8ba5-4f31-9dd2-97d09a289360,1725453807,biweekly,monday,1725453807,16,enabled,c398bb9d-8ba5-4f31-9dd2-97d09a289360,16


In [41]:
allowance_payment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2137 entries, 0 to 2136
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   uuid              2137 non-null   object
 1   creation_date     2137 non-null   int64 
 2   frequency         2137 non-null   object
 3   day               2137 non-null   object
 4   updated_at        2137 non-null   int64 
 5   next_payment_day  2137 non-null   int64 
 6   status            2137 non-null   object
 7   user_id           2137 non-null   object
 8   payment_date      2137 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 150.4+ KB


In [42]:
allowance_payment_df.nunique()

uuid                2124
creation_date       2123
frequency              4
day                   10
updated_at          1455
next_payment_day      31
status                 2
user_id             2124
payment_date          31
dtype: int64

In [43]:
incosistency_disabled_values = allowance_payment_df[(allowance_payment_df['status'] == 'disabled') & (allowance_payment_df['payment_date'].notna())]
incosistency_disabled_values

,uuid,creation_date,frequency,day,updated_at,next_payment_day,status,user_id,payment_date
1239,e359dbab-86cf-4576-8d16-3c08df3f6a41,1731781965,daily,daily,1732652076,24,disabled,e359dbab-86cf-4576-8d16-3c08df3f6a41,22
1508,38f33888-4f29-4dc5-80bc-15650c93b32b,1732237013,biweekly,friday,1733019160,13,disabled,38f33888-4f29-4dc5-80bc-15650c93b32b,6
1708,10ce6301-64f6-49bb-989c-fea0d3d5334f,1730301821,biweekly,wednesday,1732639227,4,disabled,10ce6301-64f6-49bb-989c-fea0d3d5334f,13
1893,7d108518-4d87-4b6a-857a-8b37772c75e6,1732323782,biweekly,monday,1733126436,2,disabled,7d108518-4d87-4b6a-857a-8b37772c75e6,2
1960,9d82a047-27a3-4841-9ffa-c85c5308ee6a,1723485527,biweekly,thursday,1732262437,22,disabled,9d82a047-27a3-4841-9ffa-c85c5308ee6a,22


Above, we can see that the payment_scheduled table contains 5 disabled users, which represents an inconsistency according to the process rule. No disabled user should have a record in the payment_scheduled table.
A possible reason for this could be an error in the process that failed to remove these users from the database.

In [44]:
df_allowance_payment_enabled = allowance_payment_df[allowance_payment_df['status'] == 'enabled']

In [45]:
discrepancies_payment = df_allowance_payment_enabled[(df_allowance_payment_enabled['next_payment_day'] != df_allowance_payment_enabled['payment_date'])].sort_values(by=['user_id','updated_at'], ascending = [False, False])
discrepancies_payment.head()

,uuid,creation_date,frequency,day,updated_at,next_payment_day,status,user_id,payment_date
1700,f8a6b2c5-e9f4-4564-97f4-caee992a4816,1728950477,biweekly,wednesday,1729434874,30,enabled,f8a6b2c5-e9f4-4564-97f4-caee992a4816,23
1569,f6f1fcb6-f21d-4bba-87bf-fd493181f415,1725302230,weekly,friday,1730723501,8,enabled,f6f1fcb6-f21d-4bba-87bf-fd493181f415,4
2021,ef83e1ca-eadf-4d49-a5db-4f324cc0b60f,1723382454,weekly,friday,1726195741,20,enabled,ef83e1ca-eadf-4d49-a5db-4f324cc0b60f,16
1557,eede9d14-bed8-419b-8d5f-5ea3cf897879,1725670519,weekly,friday,1731457838,15,enabled,eede9d14-bed8-419b-8d5f-5ea3cf897879,13
1752,eab4d50f-2a17-4f07-b462-53be0e54f6c6,1727740591,weekly,tuesday,1733198086,10,enabled,eab4d50f-2a17-4f07-b462-53be0e54f6c6,7


In [46]:
discrepancies_payment.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53 entries, 1700 to 1568
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   uuid              53 non-null     object
 1   creation_date     53 non-null     int64 
 2   frequency         53 non-null     object
 3   day               53 non-null     object
 4   updated_at        53 non-null     int64 
 5   next_payment_day  53 non-null     int64 
 6   status            53 non-null     object
 7   user_id           53 non-null     object
 8   payment_date      53 non-null     int64 
dtypes: int64(4), object(5)
memory usage: 4.1+ KB


In [47]:
discrepancies_payment.nunique()

uuid                52
creation_date       52
frequency            4
day                  9
updated_at          52
next_payment_day    23
status               1
user_id             52
payment_date        24
dtype: int64

- By examining the *next_payment_day* and *payment_day* fields, we observe that only 52 users have discrepancies in their payment fields for 2124. Since we have already confirmed that the *allowance_backend* table is not being updated correctly, we can infer that the same issue is occurring with the *payment_scheduled* table, which could result in significant losses for the company.

- Regarding the 52 divergent cases, a more thorough investigation by the backend team is necessary to better understand the cause of this inconsistency in the values.

### Comparative Between Allowance Event & Payment Scheduled Backend

In [48]:
event_payment = pd.merge(allowance_events_df_last_updated, payment_schedule_df, left_on='user_id', right_on='user_id', how='inner')

logger.info('Merged Allowance Events and Payment Schedule Backend data')

event_payment.head()

2025-01-15 03:50:25.574 | INFO     | __main__:<module>:3 - Merged Allowance Events and Payment Schedule Backend data


,user_id,event_timestamp,event_name,allowance_scheduled_frequency,allowance_scheduled_day,allowance_amount,next_payment_event,payment_date
0,17df1db7-0a6a-4661-90d6-949b601ab02b,1722000877,allowance.created,biweekly,monday,20,16,5
1,fb835932-ac1e-45a7-8706-545805841d0c,1722015013,allowance.created,biweekly,friday,10,13,9
2,64dd086c-5c33-4af1-a621-0936f01a59bc,1722075730,allowance.created,biweekly,friday,20,13,9
3,431a62c4-30ae-405d-95d2-fb7b8284ac4f,1722144030,allowance.created,biweekly,thursday,5,12,22
4,d93f143d-830e-4364-88c1-04cadd36b354,1722153754,allowance.created,biweekly,friday,20,13,13


In [49]:
event_payment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2134 entries, 0 to 2133
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   user_id                        2134 non-null   object
 1   event_timestamp                2134 non-null   int64 
 2   event_name                     2134 non-null   object
 3   allowance_scheduled_frequency  2134 non-null   object
 4   allowance_scheduled_day        2134 non-null   object
 5   allowance_amount               2134 non-null   int64 
 6   next_payment_event             2134 non-null   int64 
 7   payment_date                   2134 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 133.5+ KB


In [50]:
event_payment.nunique()

user_id                          2121
event_timestamp                  2121
event_name                          2
allowance_scheduled_frequency       4
allowance_scheduled_day            10
allowance_amount                   39
next_payment_event                 15
payment_date                       31
dtype: int64

In [51]:
event_payment[(event_payment['next_payment_event'] != event_payment['payment_date'])].info()

<class 'pandas.core.frame.DataFrame'>
Index: 1085 entries, 0 to 2123
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   user_id                        1085 non-null   object
 1   event_timestamp                1085 non-null   int64 
 2   event_name                     1085 non-null   object
 3   allowance_scheduled_frequency  1085 non-null   object
 4   allowance_scheduled_day        1085 non-null   object
 5   allowance_amount               1085 non-null   int64 
 6   next_payment_event             1085 non-null   int64 
 7   payment_date                   1085 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 76.3+ KB


In [52]:
event_payment[(event_payment['next_payment_event'] != event_payment['payment_date'])].nunique()

user_id                          1078
event_timestamp                  1078
event_name                          2
allowance_scheduled_frequency       4
allowance_scheduled_day             9
allowance_amount                   33
next_payment_event                 15
payment_date                       31
dtype: int64

- As confirmation, we can observe that the payment_scheduled table is also not being updated correctly, as we have numerous cases of data inconsistencies in the payment fields.

---

<div style="text-align: center;">
<span style="font-size:40px;">Conclusion</span>
</div>

1. Through our exploratory analysis, we identified several inconsistencies in the data, the most important and significant being the failure to update the payment field in the backend tables, which could cause serious harm to the company.
 
    - We also observed the presence of duplicate data in the *payment_scheduled* table, which contributes to the process inefficiencies;

    - The presence of disabled users in the *payment_scheduled* table, another critical issue requiring immediate attention;

    - Inconsistency between the frequency and day fields in the *allowance_backend* table for a specific user, which could indicate a failure at some point in the backend process;

    - Inconsistencies between the payment fields in the backend tables.


2. Based on these findings, we can conclude that certain steps in the process must be reviewed, especially the updating of the payment field in the backend tables according to the current day. Additionally, attention must be given to the inconsistencies observed in other parts of the process, and corrective action should be taken to ensure better-structured and reliable data.